<a href="https://colab.research.google.com/github/Gourav574/AI-Teaching-Assistant-for-Visual-Learning-/blob/main/Copy_of_Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Libraries (Colab)

In [ ]:
!pip install sentence-transformers
!pip install transformers
!pip install faiss-cpu
!pip install pdfplumber
!pip install python-docx
!pip install torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 85.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 5.3 MB/s eta 0:00:00


Upload Your PDF

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving Offer_Letter_Artificial_Intelligence_Intern_(Entry_Level) (1).pdf to Offer_Letter_Artificial_Intelligence_Intern_(Entry_Level) (1).pdf


Extract Text

In [ ]:
import pdfplumber
from docx import Document

def extract_text(file_path):
    if file_path.endswith(".pdf"):
        text = ""
        with pdfplumber.open(file_path) as pdf:
            for page in pdf.pages:
                t = page.extract_text()
                if t:
                    text += t + "\n"
        return text

    elif file_path.endswith(".docx"):
        doc = Document(file_path)
        return "\n".join(p.text for p in doc.paragraphs)


Chunk Text

In [ ]:
def chunk_text(text, chunk_size=400):
    words = text.split()
    return [" ".join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]


Create Embeddings + FAISS

In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

all_chunks = []

for file in uploaded.keys():
    text = extract_text(file)
    chunks = chunk_text(text)
    all_chunks.extend(chunks)

embeddings = embedding_model.encode(all_chunks)
embeddings = np.array(embeddings).astype("float32")

dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

id_to_chunk = {i: chunk for i, chunk in enumerate(all_chunks)}

print("FAISS ready ✅")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

FAISS ready ✅


Load FREE Hugging Face LLM

In [ ]:
from transformers import pipeline

qa_pipeline = pipeline(
    "text2text-generation",
    model="google/flan-t5-base",
    max_length=256
)


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


Ask Questions (FULL RAG)

In [ ]:
def answer_question(question, top_k=3):
    q_embedding = embedding_model.encode([question]).astype("float32")

    _, indices = index.search(q_embedding, top_k)
    # Filter out -1 indices which indicate no result found by FAISS
    valid_indices = [i for i in indices[0] if i != -1]
    context = "\n".join(id_to_chunk[i] for i in valid_indices)

    prompt = f"""
Answer the question based only on the context below.

Context:
{context}

Question:
{question}

Answer:
"""

    result = qa_pipeline(prompt)
    return result[0]["generated_text"]

Test It

In [ ]:
print(answer_question("What are the working hours?"))
print(answer_question("How many paid leaves are allowed?"))
print(answer_question("Is work from home allowed?"))

Token indices sequence length is longer than the specified maximum sequence length for this model (527 > 512). Running this sequence through the model will result in indexing errors


Flexible Roles & Responsibilities: • Participate in virtual training sessions, live projects, and assignments. • Collaborate with mentors and peers in project-based learning. • Submit project work and reports within deadlines. • Actively engage in discussions, reviews, and evaluations.
0
Yes
